In [3]:
# Read simulator images from udacity training set
import csv
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

def process_image(img_path, size=(32,32)):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return img
    

def read_data(batch_size):
    """
    Generator function to load driving logs and input images.
    """
    while 1:
        with open('data/driving_log.csv') as csvfile:
            reader = csv.DictReader(csvfile)
            count = 0
            # Create training set
            images = np.empty([0, 160, 320, 3], dtype=float)
            measurements = np.empty([0, ], dtype=float)
            
            try:
                for row in reader:
                    if count < int(batch_size/2):
                        image = process_image('data/'+ row['center'])
                        flip_image = np.fliplr(image)
                        steering_angle = float(row['steering'])
                        images = np.append(images, np.array([image, flip_image]), axis=0)
                        measurements = np.append(measurements, np.array([steering_angle, -steering_angle]), axis=0)
                        count += 1
                    else:
                        yield images, measurements
                        
                    if count < int(batch_size/2):
                        image = process_image('data/'+ row['left'])
                        flip_image = np.fliplr(image)
                        steering_angle = float(row['steering'] + 0.2)
                        images = np.append(images, np.array([image, flip_image]), axis=0)
                        measurements = np.append(measurements, np.array([steering_angle, -steering_angle]), axis=0)
                        count += 1
                    else:
                        yield images, measurements
                    
                    if count < int(batch_size/2):
                        image = process_image('data/'+ row['right'])
                        flip_image = np.fliplr(image)
                        steering_angle = float(row['steering'] - 0.2)
                        images = np.append(images, np.array([image, flip_image]), axis=0)
                        measurements = np.append(measurements, np.array([steering_angle, -steering_angle]), axis=0)
                        count += 1
                    else:
                        yield (images, measurements)
            except StopIteration:
                pass

def read_data2(batch_size):
    """
    Generator function to load driving logs and input images.
    """
    while 1:
        with open('data/driving_log.csv') as driving_log_file:
            driving_log_reader = csv.DictReader(driving_log_file)
            count = 0
            inputs = []
            targets = []
            try:
                for row in driving_log_reader:
                    image = process_image('data/'+ row['center'])
                    flip_image = np.fliplr(image)
                    steering_angle = float(row['steering'])
                    if count == 0:
                        inputs = np.empty([0, 160, 320, 3], dtype=float)
                        targets = np.empty([0, ], dtype=float)
                    if count < int(batch_size/2):
                        inputs = np.append(inputs, np.array([image, flip_image]), axis=0)
                        targets = np.append(targets, np.array([steering_angle, -steering_angle]), axis=0)
                        count += 1
                    else:
                        yield inputs, targets
                        count = 0
            except StopIteration:
                pass

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Cropping2D
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Lambda
from keras.layers.noise import GaussianNoise
from keras.layers.noise import GaussianDropout


model = Sequential()
# 60 pixel from top and 20 pixels from bottom of the image was cropped
model.add(Cropping2D(cropping=((60, 25), (0, 0)), input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: (x / 255.0))) # normalize the input to 0 and 1
model.add(GaussianNoise(0.1)) #additive normal noise with 0.1 STD to prevent overfitting

# I incorporated the nvidia netowrk
model.add(Conv2D(24, (5, 5), padding="valid", strides=(2, 2),
                 activation="relu", kernel_initializer="glorot_normal"))  # 24x36x158
model.add(Conv2D(36, (5, 5), padding="valid", strides=(2, 2),
                 activation="relu", kernel_initializer="glorot_normal"))  # 36x16x77
model.add(Conv2D(48, (5, 5), padding="valid", strides=(2, 2),
                 activation="relu", kernel_initializer="glorot_normal"))  # 48x6x37
# since the origianl image was slightly bigger than the one used by nividia

# I changed the filter size of layer 4 tom atch the features height at layer 5
model.add(Conv2D(64, (4, 4), padding="valid", activation="relu",
                 kernel_initializer="glorot_normal"))  # 64x3x34
model.add(Conv2D(64, (3, 3), padding="valid", activation="relu",
                 kernel_initializer="glorot_normal"))  # 64x1x32

model.add(Flatten())
# multiplicative normal noise with mean 1 and STD around 0.5 to prevent overfitting
model.add(GaussianDropout(0.25))

model.add(Dense(100, activation="relu", kernel_initializer="glorot_normal"))
model.add(Dense(50, activation="relu", kernel_initializer="glorot_normal"))
model.add(Dense(10, activation="relu", kernel_initializer="glorot_normal"))

model.add(Dense(1, kernel_initializer="glorot_normal"))

model.compile(loss='mse', optimizer='adam')
history_object = model.fit_generator(read_data2(128),
                        steps_per_epoch=100,
                        epochs=25,
                        verbose=2)

model.save('model.h5')


Epoch 1/25
613s - loss: 0.0163
Epoch 2/25


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/mrasquinha/anaconda3/anaconda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/mrasquinha/anaconda3/anaconda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mrasquinha/anaconda3/anaconda/lib/python3.6/site-packages/keras/engine/training.py", line 612, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-3-5d2800957036>", line 73, in read_data2
    image = process_image('data/'+ row['center'])
  File "<ipython-input-3-5d2800957036>", line 10, in process_image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.error: /Users/travis/miniconda3/conda-bld/opencv_1492074520667/work/opencv-3.2.0/modules/imgproc/src/color.cpp:9716: error: (-215) scn == 3 || scn == 4 in function cvtColor




ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None

In [ ]:
from IPython.display import Image, display, SVG
from keras.utils.vis_utils import model_to_dot

# Show the model in ipython notebook
figure = SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))
display(figure)

**Behavioral Cloning Project**

The goals / steps of this project are the following:
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report


#### Files Submitted & Code Quality

*1. Submission includes all required files and can be used to run the simulator in autonomous mode*

My project includes the following files:
* Model.ipynb containing the script to create and train the model
* drive.py for driving the car in autonomous mode; Changed the speed to 15 from the default udacity code
* model.h5 containing a trained convolution neural network
* writeup_report.md or writeup_report.pdf summarizing the results

*2. Submission includes functional code*
Using the Udacity provided simulator and my drive.py file, the car can be driven autonomously around the track by executing
```sh
python drive.py model.h5
```

*3. Submission code is usable and readable*
The Model.ipynb file contains the code for training and saving the convolution neural network. The file shows the pipeline I used for training and validating the model, and it contains comments to explain how the code works.

#### Model Architecture and Training Strategy

*1. An appropriate model architecture has been employed*

My model is a slightly modified version of the Nvidia architecture described in class (http://images.nvidia.com/content/tegra/automotive/images/ 2016/solutions/pdf/end-to-end-dl-using-px.pdf).
Figure 4 and "section 4 Network Architecture" provided all the details for the base implementation. I expermiented with different max pooloing   and dropout  layers. Finally settled on a single additional batch normalization layer after the first Convolution layer.

I also used the cropping layer + normalization layer as described in the lectures. This is done before the convolution layers.

*2. Attempts to reduce overfitting in the model*

Experimented with dropout layers. The best accuracy was with the additional batch norm layer.

The model was trained and validated on different data sets to ensure that the model was not overfitting. The model was tested by running it through the simulator and ensuring that the vehicle could stay on the track.

*3. Model parameter tuning*

The model used an adam optimizer, so the learning rate was not tuned manually.

*4. Appropriate training data*

I eventually used the images provided. Center+left+right camera images. I flipped some randomly picked images as can be seen in the graph after  cnt 8000. I set the validation split to be 25%. I also used some images from training the model using the simulator. I have appended the images I have generated to the end of the driving_log.csv. The simulator does not have a good response on my laptop and hence you can see the much bigger steering angles in the plot for x_val > 8000. 

I converted the images to RGB as a preprocessing step. I have also tried to use cv2 on a second set of images to visualize the effects. I was not sure how to get intermediate outputs from keras.

#### Model Architecture and Training Strategy

*1. Solution Design Approach*
1. Try to collect training images using simulator
2. Used udacity training data
3. Crop and normalize layers using keras
4. Nvidia deep learning network model
5. Experiment with different batch norm/max pooling/drop out layers. Captured model.h5 files and moitored validation accuracy.
6. Tweaked the steering angle in the original training data and tried the network.
7. Picked the best model that runs on the simulator in auto mode

*2. Final Model Architecture*
Described above

*3. Creation of the Training Set & Training Process*
Collecting my own images did not work very well. I have used a few of them for training the network.

I tried the use the fit generator aproach after reading this really amazing quora answer.
https://www.quora.com/Intuitively-how-does-mini-batch-size-affect-the-performance-of-stochastic-gradient-descent
However the smaller batch size, greater number of epochs was taking too long to run and did not yeild a higer accuracy in reasonable time. 
Thankfully while using the entire X_train set, I did not run into memory issues and this approach seemed to work.